# Downsampling and BIDS conversion

## Introduction

In this section we will demonstrate how to prepare the data for further analysis. As the FieldLine data are sampled at 5000 Hz the first step is to downsample to 1000 Hz. Subsequenctly we will convert the data to MEG BIDS (Niso et al., 2018). BIDS (Brain Imaging Data Structure) is a standardised format for organising and describing neuroimaging datasets. It ensures that the meta data are present and that the file structure is consistent over data sets. 

## Preparation

Import the required modules:

In [ ]:
import os

import numpy as np
import pandas as pd

import mne
from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids,
)

### File overview

The chapter relies on the input files:
~~~
<ROOT>/Rec15.fif
~~~ 
and generate the out files:
~~~
<ROOT>/Rec15_fs.fif
<ROOT>/Rec15_event.fif

<ROOT/BIDSRec15/participants.json
<ROOT/BIDSRec15/participants.tsv
<ROOT/BIDSRec15/sub-01/ses-01/sub-01_ses-01_scans.tsv
<ROOT/BIDSRec15/dataset_description.json
<ROOT/BIDSRec15/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json
<ROOT/BIDSRec15/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_channels.tsv
<ROOT/BIDSRec15/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_events.json
<ROOT/BIDSRec15/meg/sub-01_ses-01_task-SpAtt_run-1_events.tsv
<ROOT/BIDSRec15/meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif
<ROOT/BIDSRec15/sub-01_ses-01_task-SpAtt_run-1_meg.json
~~~

### Loading the data

The FieldLine data are stored in FIF format which is a binary file structure with embedded lables.

As a first step we will load the data. Make sure the data_data path is set to where the data are stored. We will also define the name of the resampled data with 'rs' added to the filename ( <a>  *_rs.fif  </a>).


In [ ]:
data_path = '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings'
file_name = 'Rec15.fif'
raw_fname = os.path.join(data_path, file_name)

raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw_resampled_fname = raw_fname.replace('.fif', f'_rs.fif')

event_fname = raw_fname.replace('.fif', f'_event.fif')

bids_folder = os.path.join(data_path, "BIDSRec15_03")

# Lowpass filtering and resampling

The data will now be resampled to 1000 Hz. Prior to resampling data will be lowpass filtered at 1000 Hz/4 = 250 Hz. The downsample is done to avoid alias problems (see Smith 1998, chapter 3). After the lowpass filtering the data are resampled to the desired frequency.

In [ ]:
desired_sfreq = 1000
current_sfreq = raw.info['sfreq']

lowpass_freq = desired_sfreq / 4.0
raw_resampled = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

raw_resampled.resample(sfreq=desired_sfreq)

##### As a final step, the resampled data will be saved

In [ ]:
raw_resampled.save(raw_resampled_fname, overwrite=True)

To verify that the data has been resampled write

In [ ]:
print(raw_resampled.info)

From now we will be working on the resampled data. Data with the 5000 Hz sample rate can he archived as they now longer will be used.

## Converting to MEG BIDS format

Read in the resampled data

In [ ]:
del raw, raw_resampled
raw = mne.io.read_raw(raw_resampled_fname)

### Finding triggers and write a FIF file

The BIDS data will include trigger information. As a first step all the events extract from the trigger channel and also included in an additional FIF file.

In [ ]:
events = mne.find_events(raw, stim_channel="di32")
mne.write_events(event_fname, events, overwrite=True)

### Naming the events according to trigger information

The trigger values will now be assigned labels. This will be dependent on specific design of the study and care should be
taken to assing informative labels.

In [ ]:
event_dict = {
    "cue_right": 1,
    "cue_left": 2,
    "trial_start":300,
    "stim_onset":400,
    "catch_onset":500,
    "dot_right": 6,
    "dot_left": 7,
    "resp": 800,
    "blk_start": 900,
    "blk_end":1000,
    "exp_end":1100,
    "cue_noresp":1300
}

### Writing the BIDS file

For the BIDS conversion several parameters needs to be defined according to the subjection and session. Subsequently the BIDS structure can be written.

In [ ]:
raw.info["line_freq"] = 50

subject = '01'
session = '01'
task = 'SpAtt'
run = '01'

bids_path = BIDSPath(
    subject=subject, 
    session=session, 
    task=task, 
    run=run, 
    datatype="meg", 
    root=bids_folder
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=event_fname,
    event_id=event_dict,
    overwrite=True,
)

As last check print the structure of the BIDS file

In [ ]:
print_dir_tree(bids_folder)

## References
Niso G, Gorgolewski KJ, Bock E, Brooks TL, Flandin G, Gramfort A, Henson RN, Jas M, Litvak V, T Moreau J, Oostenveld R, Schoffelen JM, Tadel F, Wexler J, Baillet S. MEG-BIDS, the brain imaging data structure extended to magnetoencephalography (2018) Sci Data 5:180110. doi: 10.1038/sdata.2018.110.

Smith SW (1998) The Scientist and Engineer's Guide to Digital Signal Processing. California Technical Pub https://www.dspguide.com/pdfbook.htm
